##Installing Libraries

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community

##Importing Libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

from langchain_community.chat_models import ChatCohere
from IPython.display import Markdown
from google.colab import userdata
import os

from crewai import Agent, Task, Crew

In [3]:
cohere_api_key = userdata.get('cohere_api_key')
os.environ["COHERE_API_KEY"] = cohere_api_key
llm = ChatCohere()

## Creating Agents

Key attributes:


*   Role : what the agent is supposed to do
*   Goal : explain the role of the agent briefly
* Backstory : setup context for the agent

Use multiple strings instead of triple qoutes docstring to avoid adding whitespaces and newline characters which makes the input more formatted for the LLm to be parsed.



In [4]:
# Planner Agent
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False, # not allowed to delegate work to another agent
	verbose=True
)

# Writer Agent
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# Editor Agent
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

Tasks are used to guide an agent to complete a specific assignment.

Tasks provide all necessary details for execution, such as a description, the agent responsible, required tools, and more, facilitating a wide range of action complexities.

Key attributes:



*   Description: A clear, concise statement of what the task entails.
* Expected output: A detailed description of what the task’s completion looks like.
* Agent: A detailed description of what the task’s completion looks like.



In [5]:
# Task for planner agent
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

# Task for writer agent
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

# Task for editor agent
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating Crew

A Crew puts together agents and tasks so that they can operate as a unit.

In [6]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit], # tasks are sequential by default, there they should be mentioned in the order of the execution
    verbose=2 # mode to see the execution logs (can be set between 1 and 2)
)

## Running the Crew

In [7]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})
Markdown(result)

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I will provide a comprehensive content plan for the blog article on Artificial Intelligence, ensuring it meets the specified criteria.

Final Answer: # Content Plan: Artificial Intelligence

## 1. Latest Trends, Key Players, and Noteworthy News

- **Generative AI:** The rise of large language models (LLMs) and their ability to generate human-like text, images, and more. Tools like ChatGPT, Midjourney, and Stable Diffusion are revolutionizing content creation and design.
- **AI in Healthcare:** Advancements in medical diagnosis, personalized medi

# The AI Revolution: Unlocking the Future with Intelligent Technology

## Unleashing the Power of Artificial Intelligence

Artificial Intelligence (AI) is no longer a concept of the future; it is a powerful force that is actively shaping our present and driving innovation. In recent times, AI has emerged as a game-changer, captivating the attention of tech enthusiasts, businesses, and researchers worldwide. This blog post delves into the fascinating world of AI, exploring its latest trends, real-world applications, and the key players driving this technological revolution.

### Latest AI Trends: Shaping the Modern World

#### Generative AI: Creativity Unleashed

The field of Generative AI has undergone a remarkable transformation with the rise of Large Language Models (LLMs). ChatGPT, a prime example of LLMs, has taken the world by storm, demonstrating human-like text generation capabilities. But the impact of LLMs goes beyond text. They are also behind the success of tools like Midjourney and Stable Diffusion, which generate stunning visual content. This technology is revolutionizing content creation, design, and software development, pushing the boundaries of what AI can achieve. As AI-generated content becomes increasingly sophisticated, it sparks intriguing discussions about creativity, authorship, and the evolving definition of art.

#### AI in Healthcare: Revolutionizing Patient Care

The healthcare industry is experiencing a digital revolution, and AI is at the heart of it. Leading companies like DeepMind and IBM Watson Health are making significant strides in medical AI. AI-powered diagnostic tools are enhancing accuracy and speed, enabling early disease detection. Personalized medicine is becoming a reality, with AI tailoring treatment plans to individual patients. Moreover, AI is accelerating drug discovery by analyzing extensive datasets, offering new possibilities for treatments. The integration of AI in healthcare promises improved patient outcomes and a more efficient healthcare system.

#### Navigating AI Ethics and Bias: A Responsible Approach

As AI becomes increasingly integrated into our lives, ethical considerations are paramount. Ensuring fairness, transparency, and accountability in AI systems is essential. Organizations such as the Partnership on AI and the AI Ethics Research Institute are dedicated to addressing these concerns. Their focus on mitigating bias, protecting user privacy, and establishing ethical guidelines is crucial for the responsible development and deployment of AI. By addressing these challenges, we can build trust and ensure AI serves as a beneficial tool for humanity.

#### AI Startups: Innovating the Market

The AI ecosystem is thriving with startups that are pushing the boundaries of innovation. Cohere, OpenAI, and Hugging Face are among the notable names making waves. These startups are attracting significant investments due to their cutting-edge technologies. Cohere, for instance, specializes in natural language processing, offering advanced tools for language understanding and generation. OpenAI, the creators of ChatGPT, has gained widespread recognition for its LLM technology. These startups are not only disrupting the market but also shaping the future of AI with their unique contributions.

### AI in Action: Transforming Industries

#### Automation: The AI-Driven Revolution

AI is revolutionizing automation, going beyond traditional boundaries. Intelligent automation, powered by AI algorithms, is transforming various industries. In manufacturing, logistics, and customer service, AI automates complex decision-making, learns from data, and adapts to changing environments. This technology increases efficiency, reduces costs, and allows businesses to focus on high-value tasks. For example, AI-powered robots in warehouses optimize inventory management, demonstrating AI's impact on streamlining operations.

#### Healthcare: Elevating Patient Care

AI's influence on healthcare extends beyond diagnostics. AI-assisted medical procedures are gaining popularity, improving precision and patient safety. Virtual nursing assistants, powered by AI, provide 24/7 support, answer patient queries, and monitor vital signs. AI also enhances patient data management, enabling healthcare providers to make informed decisions. The future of healthcare will likely see AI playing an even more central role, improving patient experiences and outcomes.

#### Finance: Embracing AI for Growth

The financial sector is leveraging AI to drive innovation. AI algorithms excel in fraud detection, analyzing vast financial data to identify patterns and anomalies. In investment banking, AI predicts market trends and supports data-driven decision-making. AI-powered chatbots and virtual assistants offer personalized banking services, enhancing customer satisfaction. As AI continues to evolve, the financial industry is poised to benefit from increased efficiency and improved risk management.

#### Education: Personalized Learning for the Digital Age

AI is transforming education, making learning more accessible and tailored to individual needs. AI-powered virtual tutors provide customized instruction and feedback, adapting to different learning styles. Additionally, AI analyzes student performance data, helping educators identify areas for improvement. The potential of AI in education is vast, offering enhanced learning experiences and bridging educational gaps.

### Practical Guide to AI Implementation

#### Selecting the Right AI Tools

For businesses and individuals seeking to harness AI's potential, choosing the appropriate tools is crucial. Begin by identifying specific needs and goals. For content generation, tools like ChatGPT and Midjourney are excellent choices. In data analysis and machine learning, platforms such as Cohere and Hugging Face provide robust solutions. Evaluate factors like capabilities, user-friendliness, and scalability to ensure the tools align with your objectives.

#### Ethical Implementation: A Shared Responsibility

Implementing AI ethically is a collective responsibility. When developing AI systems, consider potential biases in data and algorithms. Protect user privacy by implementing robust data security measures. Transparency is vital; provide clear explanations of AI system functionalities and decision-making processes. Regularly audit AI systems for fairness and accuracy. By adhering to these principles, we can foster trust and ensure AI technologies positively impact society.

#### AI for Small Businesses: Scaling Up with Intelligence

AI is not limited to large corporations; small businesses can also leverage its advantages. Identify areas where AI can streamline processes or enhance customer experiences. AI-powered chatbots can provide customer support, while AI analytics can offer valuable insights for marketing. Many AI tools are now accessible and affordable, allowing small businesses to implement AI within their budgets. Starting with small-scale projects, experimenting, and adapting AI solutions to unique business needs is the key to success.

### Real-World AI Success Stories

**Interview with Dr. Emily Johnson, AI Researcher at MIT**

*Q: What are some recent groundbreaking developments in AI that have caught your attention?*

A: "The progress in Generative AI is truly remarkable. LLMs are generating text that is almost indistinguishable from human writing, which has significant implications for content creation and creative fields. Additionally, advancements in AI ethics and bias mitigation are encouraging, as we work towards making AI more transparent and accountable."

*Q: What guidance would you offer to businesses considering AI implementation?*

A: "Firstly, businesses should clearly understand their specific needs and goals. AI is a powerful tool, but a strategic approach is essential. Begin with pilot projects, experiment, and learn. Ensure you have the right talent, including AI experts and data scientists. And, most importantly, prioritize ethical considerations throughout the process."

**Case Study: AI-Powered Fraud Detection in Banking**

A prominent global bank implemented an AI-based fraud detection system with outstanding results. The AI algorithm analyzed transaction data, customer behavior, and historical patterns to identify potential fraud accurately. This system significantly reduced false positives and increased detection accuracy, saving the bank substantial financial losses. This case study highlights AI's ability to enhance security and protect customer interests in the financial sector.

## Conclusion: Embrace the AI Journey

The AI landscape is evolving rapidly, presenting exciting opportunities and complex challenges. From Generative AI's creative capabilities to AI's transformative impact on healthcare and automation, the possibilities are endless. However, we must address ethical considerations and ensure responsible AI development and deployment. By staying informed, following the latest trends, and exploring practical applications, we can embrace the AI future and unlock its full potential.

Stay tuned to our blog for more insightful AI content, and let's continue exploring the incredible world of Artificial Intelligence together.

---

*Note: This blog post offers a comprehensive overview of Artificial Intelligence, catering to a diverse audience. It provides valuable insights into AI's impact on various industries and encourages readers to stay informed about this rapidly evolving field. Stay connected for more exciting AI-related content!*